In [1]:
#import dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
import sqlite3
from sqlite3 import Error

In [2]:
#create connections
conn = sqlite3.connect('db\wine_data.sqlite')
c = conn.cursor()


In [3]:
#read all records less than 100 dollars and have over 10 records. Leaves 97% of records
df = pd.read_sql('Select * from wine_data where price < 100 and price in (select price from wine_data group by price having count(price) > 10)', conn)
df.head(1)

,level_0,index,country,description,rating,price,province,title,variety,winery,color,countryID,varietyID,colorID,provinceID,wineryID
0,0,0,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,red,0,0,0,0,0


In [4]:
#check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96640 entries, 0 to 96639
Data columns (total 16 columns):
level_0        96640 non-null int64
index          96640 non-null int64
country        96640 non-null object
description    96640 non-null object
rating         96640 non-null int64
price          96640 non-null float64
province       96640 non-null object
title          96640 non-null object
variety        96640 non-null object
winery         96640 non-null object
color          96640 non-null object
countryID      96640 non-null int64
varietyID      96640 non-null int64
colorID        96640 non-null int64
provinceID     96640 non-null int64
wineryID       96640 non-null int64
dtypes: float64(1), int64(8), object(7)
memory usage: 11.8+ MB


In [5]:
#create the features dataframe and delete the unwanted columns
features_df = df.copy()
del features_df['price']
del features_df['index']
del features_df['description']
del features_df['title']
del features_df['province']
del features_df['winery']
del features_df['country']
del features_df['color']
del features_df['variety']
del features_df['provinceID']
del features_df['wineryID']
del features_df['level_0']

#verify the desired columns output
features_df.head(1)

,rating,countryID,varietyID,colorID
0,87,0,0,0


In [6]:
#verify the values are correct
df.head(1)

,level_0,index,country,description,rating,price,province,title,variety,winery,color,countryID,varietyID,colorID,provinceID,wineryID
0,0,0,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,red,0,0,0,0,0


In [7]:
#create X and y arrays from the dataset using the .values command
X = features_df.values
y = df['price'].values

In [8]:
#create the test and train datasets
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size = 0.25, shuffle = True)

In [9]:
#select the model and tune the hyperparameters 
model = ensemble.GradientBoostingRegressor(
    n_estimators = 300, #how many decision trees to build
    learning_rate = 0.9, #controls rate at which additional decision trees influes overall prediction
    max_depth = 6, 
    min_samples_split = 21,
    min_samples_leaf = 19, 
    max_features = 0.9,
    loss = 'huber'
)

In [10]:
#fit the model using the training data
model.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.9, loss='huber', max_depth=6,
             max_features=0.9, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=19, min_samples_split=21,
             min_weight_fraction_leaf=0.0, n_estimators=300,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [11]:
#Evaluate Results accuracy using Mean Absolute Error
mse = mean_absolute_error(y_train, model.predict(X_train))
print("Training set mean absolute error: %.2f" % mse)

mse = mean_absolute_error(y_test, model.predict(X_test))
print("Test set mean absolute error %2.f" %mse)

Training set mean absolute error: 9.19
Test set mean absolute error  9


In [12]:
#output the predictions
y_pred = model.predict(X_test[:5]).round()
y_pred

array([33., 57., 38., 28., 24.])

In [13]:
#output the actual value
y_test[:5]

array([18., 44., 50., 42., 26.])

In [14]:
#output the R2 model score
model.score(X_test, y_test)

0.5111820331972667

In [ ]:
#save the model
joblib.dump(model, 'xbgWinePrice.pkl')

In [74]:
#load the model
job_model = joblib.load("xbgWinePrice.pkl")

In [75]:
#verify the loaded model
job_model.predict(X_test[:5]).round()

array([37., 19., 26., 36., 27.])

In [15]:
#create function
def predict_price(rating, country, variety, color):
    '''
    takes the rating, country, variety, color and grabs the corresponding ID value from the 
    database to pass into the model
    '''
    
    dfco = df.loc[(df.country == country)][:1]
    dfv = df.loc[(df.variety ==variety)][:1]
    dfc = df.loc[(df.color == color)][:1]

    coid = list(dfco['countryID'])
    vid = list(dfv['varietyID'])
    cid = list(dfv['colorID'])   

    dft = pd.DataFrame({'rating':rating,
                        "countryID":coid,
                        "varietyID":vid,
                        "colorID":cid,})
    
    return model.predict(dft).round()
                      

In [16]:
predict_price(87, 'Portugal','Portuguese Red','red')

array([14.])

In [17]:
g = list(df['price'][:1])
print("Actual Price: ")
g

Actual Price: 


[15.0]